In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import math

In [3]:
# load training and test datasets
test_set = pd.read_csv('../input/test_set.csv', parse_dates=[2,])
train_set = pd.read_csv('../input/train_set.csv', parse_dates=[2,])
tube              = pd.read_csv('../input/tube.csv', parse_dates=True)
specs             = pd.read_csv('../input/specs.csv', parse_dates=[2,])
bill_of_materials = pd.read_csv('../input/bill_of_materials.csv', parse_dates=True)
comp_adaptor      = pd.read_csv('../input/comp_adaptor.csv', parse_dates=[2,])
comp_boss         = pd.read_csv('../input/comp_boss.csv', parse_dates=[2,])
comp_elbow        = pd.read_csv('../input/comp_elbow.csv', parse_dates=[2,])
comp_float        = pd.read_csv('../input/comp_float.csv', parse_dates=[2,])
comp_hfl          = pd.read_csv('../input/comp_hfl.csv', parse_dates=[2,])
comp_nut          = pd.read_csv('../input/comp_nut.csv', parse_dates=[2,])
comp_other        = pd.read_csv('../input/comp_other.csv', parse_dates=[2,])
comp_sleeve       = pd.read_csv('../input/comp_sleeve.csv', parse_dates=[2,])
comp_straight     = pd.read_csv('../input/comp_straight.csv', parse_dates=[2,])
comp_tee          = pd.read_csv('../input/comp_tee.csv', parse_dates=[2,])
comp_threaded     = pd.read_csv('../input/comp_threaded.csv', parse_dates=[2,])
components        = pd.read_csv('../input/components.csv', parse_dates=[2,])
test_set          = pd.read_csv('../input/test_set.csv', parse_dates=[2,])
train_set         = pd.read_csv('../input/train_set.csv', parse_dates=[2,])
tube_end_form     = pd.read_csv('../input/tube_end_form.csv', parse_dates=True)
tube              = pd.read_csv('../input/tube.csv', parse_dates=True)
type_component    = pd.read_csv('../input/type_component.csv', parse_dates=True)
type_connection   = pd.read_csv('../input/type_connection.csv', parse_dates=True)
type_end_form     = pd.read_csv('../input/type_end_form.csv', parse_dates=True)


In [4]:
labels = train_set.cost.values
label_log = np.log1p(labels)

In [5]:
BoF_1 = bill_of_materials[['tube_assembly_id','component_id_1','quantity_1']]
BoF_2 = bill_of_materials[['tube_assembly_id','component_id_2','quantity_2']]
comp_adaptor = pd.merge(comp_adaptor,type_component,on='component_type_id',how='inner')
comp_boss = pd.merge(comp_boss,type_component,on='component_type_id',how='inner')
comp_elbow = pd.merge(comp_elbow,type_component,on='component_type_id',how='inner')
comp_float = pd.merge(comp_float,type_component,on='component_type_id',how='inner')
comp_hfl = pd.merge(comp_hfl,type_component,on='component_type_id',how='inner')
comp_nut  = pd.merge(comp_nut,type_component,on='component_type_id',how='inner')
comp_sleeve  = pd.merge(comp_sleeve,type_component,on='component_type_id',how='inner')
comp_straight  = pd.merge(comp_sleeve,type_component,on='component_type_id',how='inner')
comp_tee  = pd.merge(comp_straight,type_component,on='component_type_id',how='inner')
comp_threaded  = pd.merge(comp_threaded,type_component,on='component_type_id',how='inner')


In [6]:
def merge_test_train_spec_tube(df):
    df = pd.merge(df,tube,on='tube_assembly_id')
    df = pd.merge(df,specs,on='tube_assembly_id')
    return df


In [7]:
train_set = merge_test_train_spec_tube(train_set)
test_set = merge_test_train_spec_tube(test_set)

In [8]:
test_set = test_set.drop(['spec10'], axis=1)
train_set = train_set.drop(['spec10'], axis=1)
bill_of_materials = bill_of_materials.drop(['component_id_5','quantity_5','component_id_6',
                                            'quantity_6', 'component_id_7','quantity_7', 'component_id_8','quantity_8'], axis=1)
comp_threaded = comp_threaded.drop(['end_form_id_3', 'connection_type_id_3','length_3','thread_size_3',
                                   'thread_pitch_3', 'nominal_size_3', 'end_form_id_4','connection_type_id_4',
                                   'length_4','thread_size_4','thread_pitch_4', 'nominal_size_4'],axis=1)

In [9]:
train_set = pd.merge(train_set,bill_of_materials,on='tube_assembly_id',how='inner')
test_set = pd.merge(test_set,bill_of_materials,on='tube_assembly_id',how='inner')

In [10]:
components = components.rename({'name':'name_components'}, axis=1)
components = components.drop(['component_type_id'], axis=1)

In [11]:
comp_adaptor  = pd.merge(comp_adaptor,components,on='component_id',how='left')
comp_boss  = pd.merge(comp_boss,components,on='component_id',how='left')
comp_elbow  = pd.merge(comp_elbow,components,on='component_id',how='left')
comp_float  = pd.merge(comp_float,components,on='component_id',how='left')
comp_hfl  = pd.merge(comp_hfl,components,on='component_id',how='left')
comp_nut  = pd.merge(comp_nut,components,on='component_id',how='left')
comp_sleeve = pd.merge(comp_sleeve,components,on='component_id',how='left')

In [12]:
comp_straight = comp_straight.drop(['name_y'],axis=1)
comp_straight = comp_straight.rename({'name_x':'name'}, axis=1)
comp_straight = pd.merge(comp_straight,components,on='component_id',how='left')

In [13]:
comp_tee = comp_tee.drop(['name_y','name_x'],axis=1)
comp_tee = pd.merge(comp_tee,components,on='component_id',how='left')

In [14]:
comp_threaded = pd.merge(comp_threaded,components,on='component_id',how='left')

In [15]:
type_end_form = type_end_form.rename({'name':'name_end_form_id'}, axis=1)

In [16]:
type_connection = type_connection.rename({'name':'name_type_connection_id'}, axis=1)

In [17]:
comp_threaded = pd.merge(comp_threaded,type_end_form,left_on='end_form_id_1',right_on='end_form_id',how='inner')
comp_threaded = comp_threaded.rename({'name_end_form_id':'name_end_form_id_1'}, axis=1)
comp_threaded = comp_threaded.drop(['end_form_id'],axis=1)
comp_threaded = pd.merge(comp_threaded,type_connection,left_on='connection_type_id_1',
                         right_on='connection_type_id',how='inner')
comp_threaded = comp_threaded.rename({'name_type_connection_id':'name_type_connection_id_1'}, axis=1)
comp_threaded = comp_threaded.drop(['connection_type_id'],axis=1)
comp_threaded = comp_threaded[['component_id', 'component_type_id', 'adaptor_angle', 'overall_length',
       'hex_size', 'end_form_id_1','name_end_form_id_1', 'connection_type_id_1','name_type_connection_id_1','length_1',
       'thread_size_1', 'thread_pitch_1', 'nominal_size_1', 'end_form_id_2',
       'connection_type_id_2', 'length_2', 'thread_size_2', 'thread_pitch_2',
       'nominal_size_2', 'unique_feature', 'orientation', 'weight', 'name',
       'name_components']]

In [18]:
comp_adaptor = pd.merge(comp_adaptor,type_end_form,left_on='end_form_id_1',right_on='end_form_id',how='inner')
comp_adaptor = comp_adaptor.rename({'name_end_form_id':'name_end_form_id_1'}, axis=1)
comp_adaptor = comp_adaptor.drop(['end_form_id'],axis=1)
comp_adaptor = pd.merge(comp_adaptor,type_connection,left_on='connection_type_id_1',
                         right_on='connection_type_id',how='inner')
comp_adaptor = comp_adaptor.rename({'name_type_connection_id':'name_type_connection_id_1'}, axis=1)
comp_adaptor = comp_adaptor.drop(['connection_type_id'],axis=1)
comp_adaptor = comp_adaptor[['component_id', 'component_type_id', 'adaptor_angle', 'overall_length',
       'end_form_id_1','name_end_form_id_1', 'connection_type_id_1','name_type_connection_id_1','length_1',
       'thread_size_1', 'thread_pitch_1', 'nominal_size_1', 'end_form_id_2',
       'connection_type_id_2', 'length_2', 'thread_size_2', 'thread_pitch_2',
       'nominal_size_2','hex_size', 'unique_feature', 'orientation', 'weight', 'name',
       'name_components']]

In [19]:
comp_boss = pd.merge(comp_boss,type_connection,left_on='connection_type_id',
                         right_on='connection_type_id',how='inner')
comp_boss = comp_boss[['component_id', 'component_type_id', 'type', 'connection_type_id', 'name_type_connection_id',
       'outside_shape', 'base_type', 'height_over_tube', 'bolt_pattern_long',
       'bolt_pattern_wide', 'groove', 'base_diameter', 'shoulder_diameter',
       'unique_feature', 'orientation', 'weight', 'name', 'name_components',
      ]]

In [20]:
comp_sleeve = pd.merge(comp_sleeve,type_connection,left_on='connection_type_id',
                         right_on='connection_type_id',how='inner')
comp_sleeve = comp_sleeve[['component_id', 'component_type_id', 'connection_type_id', 'name_type_connection_id','length',
       'intended_nut_thread', 'intended_nut_pitch', 'unique_feature',
       'plating', 'orientation', 'weight', 'name', 'name_components']]

In [21]:
comp_straight = pd.merge(comp_straight,type_connection,left_on='connection_type_id',
                         right_on='connection_type_id',how='inner')
comp_straight = comp_straight[['component_id', 'component_type_id', 'connection_type_id','name_type_connection_id', 'length',
       'intended_nut_thread', 'intended_nut_pitch', 'unique_feature',
       'plating', 'orientation', 'weight', 'name', 'name_components'
       ]]

In [22]:
comp_tee = pd.merge(comp_tee,type_connection,left_on='connection_type_id',
                         right_on='connection_type_id',how='inner')
comp_tee = comp_tee[['component_id', 'component_type_id', 'connection_type_id','name_type_connection_id', 'length',
       'intended_nut_thread', 'intended_nut_pitch', 'unique_feature',
       'plating', 'orientation', 'weight', 'name', 'name_components'
       ]]

In [23]:
bill_of_materials = bill_of_materials.drop(['component_id_3', 'quantity_3', 'component_id_4', 'quantity_4'],axis=1)

In [24]:
BoF_1 = bill_of_materials.drop(['component_id_2', 'quantity_2'],axis=1)
BoF_2 = bill_of_materials.drop(['component_id_1', 'quantity_1'],axis=1)

In [25]:
BoF_1_adaptor = pd.merge(BoF_1, comp_adaptor, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_boss = pd.merge(BoF_1, comp_boss, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_elbow = pd.merge(BoF_1, comp_elbow, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_float = pd.merge(BoF_1, comp_float, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_nut = pd.merge(BoF_1, comp_nut, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_other = pd.merge(BoF_1, comp_other, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_sleeve = pd.merge(BoF_1, comp_sleeve, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_straight = pd.merge(BoF_1, comp_straight, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_tee = pd.merge(BoF_1, comp_tee, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_threaded = pd.merge(BoF_1, comp_threaded, left_on='component_id_1',right_on='component_id',how='right')

In [26]:
def concat_ordered_columns(frames, ax):
    columns_ordered = []
    for frame in frames:
        columns_ordered.extend(x for x in frame.columns if x not in columns_ordered)
    final_df = pd.concat(frames,ignore_index=True,axis=ax)    
    return final_df[columns_ordered]       


dfs = [BoF_1_adaptor,BoF_1_boss,BoF_1_elbow,BoF_1_float,BoF_1_float,BoF_1_nut,BoF_1_other,
       BoF_1_sleeve,BoF_1_straight,BoF_1_tee,BoF_1_threaded]
full_1 = concat_ordered_columns(dfs,0)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [27]:
full_1 = full_1[pd.notnull(full_1['tube_assembly_id'])]

In [28]:
BoF_2_adaptor = pd.merge(BoF_2, comp_adaptor, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_boss = pd.merge(BoF_2, comp_boss, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_elbow = pd.merge(BoF_2, comp_elbow, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_float = pd.merge(BoF_2, comp_float, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_nut = pd.merge(BoF_2, comp_nut, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_other = pd.merge(BoF_2, comp_other, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_sleeve = pd.merge(BoF_2, comp_sleeve, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_straight = pd.merge(BoF_2, comp_straight, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_tee = pd.merge(BoF_2, comp_tee, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_threaded = pd.merge(BoF_2, comp_threaded, left_on='component_id_2',right_on='component_id',how='right')

In [29]:
dfs = [BoF_2_adaptor,BoF_2_boss,BoF_2_elbow,BoF_2_float,BoF_2_float,BoF_2_nut,BoF_2_other,
       BoF_2_sleeve,BoF_2_straight,BoF_2_tee,BoF_2_threaded]
full_2 = concat_ordered_columns(dfs,0)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [30]:
train_set = pd.merge(train_set, full_1, on='tube_assembly_id',how='left')
train_set = pd.merge(train_set, full_2, on='tube_assembly_id',how='left')

test_set = pd.merge(test_set, full_1, on='tube_assembly_id',how='left')
test_set = pd.merge(test_set, full_2, on='tube_assembly_id',how='left')

In [31]:
train_set = train_set.drop_duplicates()
test_set = test_set.drop_duplicates()

In [32]:
train_set['year'] = train_set.quote_date.dt.year
train_set['month'] = train_set.quote_date.dt.month
train_set['day'] = train_set.quote_date.dt.day

In [33]:
test_set['quote_date'] = pd.to_datetime(test_set['quote_date'])

In [34]:
test_set['quote_date'] = pd.to_datetime(test_set['quote_date'])

test_set['year'] = test_set.quote_date.dt.year
test_set['month'] = test_set.quote_date.dt.month
test_set['day'] = test_set.quote_date.dt.day

In [35]:

idx = test_set.id.values.astype(int)
test_set = test_set.drop(['id', 'tube_assembly_id', 'quote_date'], axis = 1)
labels = train_set.cost.values


train_set = train_set.drop(['quote_date', 'cost', 'tube_assembly_id'], axis = 1)

In [36]:
train_set.shape,test_set.shape

((30213, 150), (30235, 150))

In [37]:
not_cat = []
for i in train_set.columns:
    if train_set[i].dtypes != 'object':
            not_cat.append(i)
not_cat    

['annual_usage',
 'min_order_quantity',
 'quantity',
 'diameter_x',
 'wall',
 'length_x',
 'num_bends',
 'bend_radius',
 'num_boss',
 'num_bracket',
 'other',
 'quantity_1_x',
 'quantity_2_x',
 'quantity_3',
 'quantity_4',
 'quantity_1_y',
 'overall_length_x',
 'length_1_x',
 'thread_size_1_x',
 'thread_pitch_1_x',
 'length_2_x',
 'thread_size_2_x',
 'thread_pitch_2_x',
 'nominal_size_2_x',
 'hex_size_x',
 'height_over_tube_x',
 'bolt_pattern_wide_x',
 'base_diameter_x',
 'shoulder_diameter_x',
 'extension_length_x',
 'thickness_x',
 'drop_length_x',
 'elbow_angle_x',
 'plug_diameter_x',
 'seat_angle_x',
 'length_y',
 'thread_pitch_x',
 'diameter_y',
 'intended_nut_thread_x',
 'intended_nut_pitch_x',
 'quantity_2_y',
 'overall_length_y',
 'length_1_y',
 'thread_size_1_y',
 'thread_pitch_1_y',
 'length_2_y',
 'thread_size_2_y',
 'thread_pitch_2_y',
 'nominal_size_2_y',
 'hex_size_y',
 'height_over_tube_y',
 'bolt_pattern_wide_y',
 'base_diameter_y',
 'shoulder_diameter_y',
 'extension_l

In [38]:
ind_categorical = []

for i in train_set.columns:
    if train_set[i].dtypes == 'object':
            values = train_set[i].value_counts().keys().tolist()
            counts = train_set[i].value_counts().tolist()

In [39]:
ind_categorical = []
name_categorical = []
for i in train_set.columns:
    if train_set[i].dtypes == 'object':
        #if train_set.columns.get_loc(i) not in not_cat:
        ind_categorical.append(train_set.columns.get_loc(i))  
        name_categorical.append(i) 

In [40]:
label_log = np.log1p(labels)

In [41]:
del BoF_1, BoF_1_adaptor,BoF_1_boss,BoF_1_elbow,BoF_1_float,BoF_1_nut,BoF_1_other,BoF_1_sleeve,BoF_1_straight,BoF_1_tee,BoF_1_threaded

In [42]:
del BoF_2, BoF_2_adaptor,BoF_2_boss,BoF_2_elbow,BoF_2_float,BoF_2_nut,BoF_2_other,BoF_2_sleeve,BoF_2_straight,BoF_2_tee,BoF_2_threaded

In [43]:
del bill_of_materials,comp_adaptor,comp_boss,comp_elbow,comp_float,comp_hfl,comp_nut,comp_other,comp_sleeve,comp_straight,comp_tee,comp_threaded,components

In [44]:
del full_1,full_2,specs,tube,tube_end_form,type_component,type_connection,type_end_form

In [45]:
del labels

In [46]:
ntrain=train_set.shape[0]
ntest=test_set.shape[0]
print(ntrain, ntest)



30213 30235


In [47]:
for col in name_categorical:
    train_set[col] = train_set[col].astype(str)
    test_set[col] = test_set[col].astype(str)


In [48]:
from sklearn.model_selection import KFold
# Some useful parameters which will come in handy later on
SEED = 4 # for reproducibility
NFOLDS = 5# set folds for out-of-fold prediction
kf = KFold(n_splits= NFOLDS, shuffle=False)
i=0
for train_index, test_index in kf.split(train_set):
    print("TRAIN:", train_index, "TEST:", test_index)



TRAIN: [ 6043  6044  6045 ... 30210 30211 30212] TEST: [   0    1    2 ... 6040 6041 6042]
TRAIN: [    0     1     2 ... 30210 30211 30212] TEST: [ 6043  6044  6045 ... 12083 12084 12085]
TRAIN: [    0     1     2 ... 30210 30211 30212] TEST: [12086 12087 12088 ... 18126 18127 18128]
TRAIN: [    0     1     2 ... 30210 30211 30212] TEST: [18129 18130 18131 ... 24168 24169 24170]
TRAIN: [    0     1     2 ... 24168 24169 24170] TEST: [24171 24172 24173 ... 30210 30211 30212]


In [77]:
pred_train = np.zeros((ntrain,) )
score = 0
from sklearn import metrics
from catboost import  Pool
from catboost import CatBoostRegressor
i = 1
for train_index, test_index in  kf.split(train_set):
    s=str(i)+"CatB.cbm"
    train_pool = Pool(train_set.iloc[train_index],label_log[train_index],cat_features=ind_categorical)
    val_pool_data = Pool(train_set.iloc[test_index],label_log[test_index],cat_features=ind_categorical)
    
    model =  CatBoostRegressor(learning_rate=0.1,
                               depth=8,
                               one_hot_max_size=10,
                               iterations=2000,
                               use_best_model=True,
                               bootstrap_type='Bernoulli',
                               subsample=1, 
                               random_seed=i,
                               sampling_unit ='Group',
                               rsm=1,
                               nan_mode='Min',
                               leaf_estimation_iterations=5,
                               fold_len_multiplier=2,
                               boosting_type='Plain')
    
    model.fit(train_pool,plot=False,eval_set=val_pool_data)
    
    model.save_model(s, format="cbm", export_parameters=False)
    pr=model.predict(train_set.iloc[test_index])
    pred_train[test_index]=pr
    err=metrics.mean_squared_error(label_log[test_index], pr)
    score += err*(1/NFOLDS) 
    print("Fold", i, "Точность работы на тестовых данных:",  err)
    i+=1
    print(s)

0:	learn: 0.8123800	test: 0.7788391	best: 0.7788391 (0)	total: 115ms	remaining: 7m 40s
1:	learn: 0.7956410	test: 0.7627397	best: 0.7627397 (1)	total: 230ms	remaining: 7m 39s
2:	learn: 0.7799183	test: 0.7480516	best: 0.7480516 (2)	total: 346ms	remaining: 7m 40s
3:	learn: 0.7645444	test: 0.7327142	best: 0.7327142 (3)	total: 437ms	remaining: 7m 16s
4:	learn: 0.7497701	test: 0.7182870	best: 0.7182870 (4)	total: 537ms	remaining: 7m 9s
5:	learn: 0.7347821	test: 0.7037148	best: 0.7037148 (5)	total: 645ms	remaining: 7m 9s
6:	learn: 0.7208204	test: 0.6900239	best: 0.6900239 (6)	total: 742ms	remaining: 7m 3s
7:	learn: 0.7076565	test: 0.6773890	best: 0.6773890 (7)	total: 831ms	remaining: 6m 54s
8:	learn: 0.6944775	test: 0.6647418	best: 0.6647418 (8)	total: 931ms	remaining: 6m 52s
9:	learn: 0.6817008	test: 0.6530222	best: 0.6530222 (9)	total: 1.03s	remaining: 6m 50s
10:	learn: 0.6697445	test: 0.6420321	best: 0.6420321 (10)	total: 1.13s	remaining: 6m 48s
11:	learn: 0.6582597	test: 0.6315374	best: 0

93:	learn: 0.3024189	test: 0.3209055	best: 0.3209055 (93)	total: 9.85s	remaining: 6m 49s
94:	learn: 0.3010520	test: 0.3197302	best: 0.3197302 (94)	total: 9.96s	remaining: 6m 49s
95:	learn: 0.2995776	test: 0.3185764	best: 0.3185764 (95)	total: 10.1s	remaining: 6m 49s
96:	learn: 0.2980642	test: 0.3175017	best: 0.3175017 (96)	total: 10.2s	remaining: 6m 48s
97:	learn: 0.2967345	test: 0.3165832	best: 0.3165832 (97)	total: 10.3s	remaining: 6m 48s
98:	learn: 0.2953556	test: 0.3154985	best: 0.3154985 (98)	total: 10.4s	remaining: 6m 48s
99:	learn: 0.2941703	test: 0.3148328	best: 0.3148328 (99)	total: 10.5s	remaining: 6m 49s
100:	learn: 0.2930799	test: 0.3141500	best: 0.3141500 (100)	total: 10.6s	remaining: 6m 48s
101:	learn: 0.2920864	test: 0.3136833	best: 0.3136833 (101)	total: 10.7s	remaining: 6m 48s
102:	learn: 0.2909910	test: 0.3129222	best: 0.3129222 (102)	total: 10.8s	remaining: 6m 48s
103:	learn: 0.2901808	test: 0.3123789	best: 0.3123789 (103)	total: 10.9s	remaining: 6m 48s
104:	learn: 0

184:	learn: 0.2415290	test: 0.2843111	best: 0.2843111 (184)	total: 20.1s	remaining: 6m 55s
185:	learn: 0.2411301	test: 0.2840801	best: 0.2840801 (185)	total: 20.3s	remaining: 6m 55s
186:	learn: 0.2409038	test: 0.2838789	best: 0.2838789 (186)	total: 20.4s	remaining: 6m 55s
187:	learn: 0.2404036	test: 0.2836542	best: 0.2836542 (187)	total: 20.5s	remaining: 6m 55s
188:	learn: 0.2402106	test: 0.2836056	best: 0.2836056 (188)	total: 20.6s	remaining: 6m 55s
189:	learn: 0.2397852	test: 0.2832844	best: 0.2832844 (189)	total: 20.7s	remaining: 6m 55s
190:	learn: 0.2394775	test: 0.2831210	best: 0.2831210 (190)	total: 20.8s	remaining: 6m 55s
191:	learn: 0.2392398	test: 0.2830150	best: 0.2830150 (191)	total: 20.9s	remaining: 6m 54s
192:	learn: 0.2387917	test: 0.2828565	best: 0.2828565 (192)	total: 21s	remaining: 6m 54s
193:	learn: 0.2385464	test: 0.2826940	best: 0.2826940 (193)	total: 21.1s	remaining: 6m 54s
194:	learn: 0.2382591	test: 0.2824987	best: 0.2824987 (194)	total: 21.3s	remaining: 6m 54s
1

275:	learn: 0.2185119	test: 0.2733012	best: 0.2733012 (275)	total: 30.1s	remaining: 6m 45s
276:	learn: 0.2183795	test: 0.2732311	best: 0.2732311 (276)	total: 30.2s	remaining: 6m 45s
277:	learn: 0.2180655	test: 0.2731420	best: 0.2731420 (277)	total: 30.3s	remaining: 6m 45s
278:	learn: 0.2178598	test: 0.2730558	best: 0.2730558 (278)	total: 30.4s	remaining: 6m 45s
279:	learn: 0.2177422	test: 0.2729829	best: 0.2729829 (279)	total: 30.5s	remaining: 6m 45s
280:	learn: 0.2175791	test: 0.2729153	best: 0.2729153 (280)	total: 30.6s	remaining: 6m 44s
281:	learn: 0.2174445	test: 0.2728739	best: 0.2728739 (281)	total: 30.7s	remaining: 6m 44s
282:	learn: 0.2167062	test: 0.2725907	best: 0.2725907 (282)	total: 30.8s	remaining: 6m 44s
283:	learn: 0.2165156	test: 0.2725030	best: 0.2725030 (283)	total: 30.9s	remaining: 6m 44s
284:	learn: 0.2163604	test: 0.2724084	best: 0.2724084 (284)	total: 31s	remaining: 6m 44s
285:	learn: 0.2162135	test: 0.2723997	best: 0.2723997 (285)	total: 31.1s	remaining: 6m 44s
2

366:	learn: 0.2022716	test: 0.2672882	best: 0.2672824 (365)	total: 40.3s	remaining: 6m 38s
367:	learn: 0.2022149	test: 0.2672951	best: 0.2672824 (365)	total: 40.4s	remaining: 6m 38s
368:	learn: 0.2021580	test: 0.2672769	best: 0.2672769 (368)	total: 40.5s	remaining: 6m 38s
369:	learn: 0.2019610	test: 0.2672390	best: 0.2672390 (369)	total: 40.6s	remaining: 6m 38s
370:	learn: 0.2015728	test: 0.2671039	best: 0.2671039 (370)	total: 40.7s	remaining: 6m 38s
371:	learn: 0.2014908	test: 0.2670693	best: 0.2670693 (371)	total: 40.8s	remaining: 6m 38s
372:	learn: 0.2014583	test: 0.2670615	best: 0.2670615 (372)	total: 40.9s	remaining: 6m 37s
373:	learn: 0.2013302	test: 0.2670468	best: 0.2670468 (373)	total: 41s	remaining: 6m 37s
374:	learn: 0.2012319	test: 0.2670267	best: 0.2670267 (374)	total: 41.1s	remaining: 6m 37s
375:	learn: 0.2009451	test: 0.2669319	best: 0.2669319 (375)	total: 41.2s	remaining: 6m 37s
376:	learn: 0.2008124	test: 0.2669387	best: 0.2669319 (375)	total: 41.3s	remaining: 6m 37s
3

...

3561:	learn: 0.0773612	test: 0.2572329	best: 0.2572319 (3557)	total: 7m 26s	remaining: 54.9s
3562:	learn: 0.0773601	test: 0.2572323	best: 0.2572319 (3557)	total: 7m 26s	remaining: 54.8s
3563:	learn: 0.0773403	test: 0.2572308	best: 0.2572308 (3563)	total: 7m 26s	remaining: 54.7s
3564:	learn: 0.0773250	test: 0.2572268	best: 0.2572268 (3564)	total: 7m 27s	remaining: 54.5s
3565:	learn: 0.0773190	test: 0.2572262	best: 0.2572262 (3565)	total: 7m 27s	remaining: 54.4s
3566:	learn: 0.0772983	test: 0.2572220	best: 0.2572220 (3566)	total: 7m 27s	remaining: 54.3s
3567:	learn: 0.0772702	test: 0.2572182	best: 0.2572182 (3567)	total: 7m 27s	remaining: 54.2s
3568:	learn: 0.0772695	test: 0.2572186	best: 0.2572182 (3567)	total: 7m 27s	remaining: 54s
3569:	learn: 0.0772544	test: 0.2572234	best: 0.2572182 (3567)	total: 7m 27s	remaining: 53.9s
3570:	learn: 0.0772510	test: 0.2572175	best: 0.2572175 (3570)	total: 7m 27s	remaining: 53.8s
3571:	learn: 0.0772363	test: 0.2572148	best: 0.2572148 (3571)	total: 7m 

3650:	learn: 0.0759443	test: 0.2571130	best: 0.2571130 (3650)	total: 7m 37s	remaining: 43.8s
3651:	learn: 0.0759356	test: 0.2571094	best: 0.2571094 (3651)	total: 7m 37s	remaining: 43.6s
3652:	learn: 0.0759043	test: 0.2570994	best: 0.2570994 (3652)	total: 7m 38s	remaining: 43.5s
3653:	learn: 0.0759036	test: 0.2570982	best: 0.2570982 (3653)	total: 7m 38s	remaining: 43.4s
3654:	learn: 0.0758994	test: 0.2571017	best: 0.2570982 (3653)	total: 7m 38s	remaining: 43.3s
3655:	learn: 0.0758702	test: 0.2570961	best: 0.2570961 (3655)	total: 7m 38s	remaining: 43.1s
3656:	learn: 0.0758595	test: 0.2570890	best: 0.2570890 (3656)	total: 7m 38s	remaining: 43s
3657:	learn: 0.0758193	test: 0.2570875	best: 0.2570875 (3657)	total: 7m 38s	remaining: 42.9s
3658:	learn: 0.0758019	test: 0.2570831	best: 0.2570831 (3658)	total: 7m 38s	remaining: 42.8s
3659:	learn: 0.0757776	test: 0.2570787	best: 0.2570787 (3659)	total: 7m 38s	remaining: 42.6s
3660:	learn: 0.0757677	test: 0.2570767	best: 0.2570767 (3660)	total: 7m 

3740:	learn: 0.0745683	test: 0.2569600	best: 0.2569471 (3736)	total: 7m 49s	remaining: 32.5s
3741:	learn: 0.0745498	test: 0.2569450	best: 0.2569450 (3741)	total: 7m 49s	remaining: 32.4s
3742:	learn: 0.0745249	test: 0.2569482	best: 0.2569450 (3741)	total: 7m 49s	remaining: 32.2s
3743:	learn: 0.0744839	test: 0.2569553	best: 0.2569450 (3741)	total: 7m 49s	remaining: 32.1s
3744:	learn: 0.0744705	test: 0.2569475	best: 0.2569450 (3741)	total: 7m 49s	remaining: 32s
3745:	learn: 0.0744532	test: 0.2569463	best: 0.2569450 (3741)	total: 7m 49s	remaining: 31.9s
3746:	learn: 0.0744277	test: 0.2569472	best: 0.2569450 (3741)	total: 7m 49s	remaining: 31.7s
3747:	learn: 0.0744207	test: 0.2569482	best: 0.2569450 (3741)	total: 7m 50s	remaining: 31.6s
3748:	learn: 0.0743915	test: 0.2569507	best: 0.2569450 (3741)	total: 7m 50s	remaining: 31.5s
3749:	learn: 0.0743912	test: 0.2569504	best: 0.2569450 (3741)	total: 7m 50s	remaining: 31.4s
3750:	learn: 0.0743679	test: 0.2569453	best: 0.2569450 (3741)	total: 7m 

3830:	learn: 0.0731884	test: 0.2569280	best: 0.2568795 (3779)	total: 8m	remaining: 21.2s
3831:	learn: 0.0731527	test: 0.2569281	best: 0.2568795 (3779)	total: 8m	remaining: 21.1s
3832:	learn: 0.0731525	test: 0.2569281	best: 0.2568795 (3779)	total: 8m	remaining: 21s
3833:	learn: 0.0731484	test: 0.2569279	best: 0.2568795 (3779)	total: 8m 1s	remaining: 20.8s
3834:	learn: 0.0731405	test: 0.2569206	best: 0.2568795 (3779)	total: 8m 1s	remaining: 20.7s
3835:	learn: 0.0731203	test: 0.2569176	best: 0.2568795 (3779)	total: 8m 1s	remaining: 20.6s
3836:	learn: 0.0731192	test: 0.2569124	best: 0.2568795 (3779)	total: 8m 1s	remaining: 20.5s
3837:	learn: 0.0731111	test: 0.2569046	best: 0.2568795 (3779)	total: 8m 1s	remaining: 20.3s
3838:	learn: 0.0731024	test: 0.2569215	best: 0.2568795 (3779)	total: 8m 1s	remaining: 20.2s
3839:	learn: 0.0730873	test: 0.2569263	best: 0.2568795 (3779)	total: 8m 1s	remaining: 20.1s
3840:	learn: 0.0730773	test: 0.2569240	best: 0.2568795 (3779)	total: 8m 1s	remaining: 19.9s

3920:	learn: 0.0719570	test: 0.2569271	best: 0.2568795 (3779)	total: 8m 12s	remaining: 9.91s
3921:	learn: 0.0719511	test: 0.2569184	best: 0.2568795 (3779)	total: 8m 12s	remaining: 9.79s
3922:	learn: 0.0719343	test: 0.2569205	best: 0.2568795 (3779)	total: 8m 12s	remaining: 9.66s
3923:	learn: 0.0719148	test: 0.2569201	best: 0.2568795 (3779)	total: 8m 12s	remaining: 9.54s
3924:	learn: 0.0719028	test: 0.2569244	best: 0.2568795 (3779)	total: 8m 12s	remaining: 9.41s
3925:	learn: 0.0719016	test: 0.2569248	best: 0.2568795 (3779)	total: 8m 12s	remaining: 9.29s
3926:	learn: 0.0718679	test: 0.2569259	best: 0.2568795 (3779)	total: 8m 12s	remaining: 9.16s
3927:	learn: 0.0718663	test: 0.2569262	best: 0.2568795 (3779)	total: 8m 12s	remaining: 9.04s
3928:	learn: 0.0718317	test: 0.2569189	best: 0.2568795 (3779)	total: 8m 13s	remaining: 8.91s
3929:	learn: 0.0718314	test: 0.2569188	best: 0.2568795 (3779)	total: 8m 13s	remaining: 8.78s
3930:	learn: 0.0717911	test: 0.2569209	best: 0.2568795 (3779)	total: 8

In [78]:
score

0.07327308111416587

In [79]:
wr=1./NFOLDS
predTest = np.zeros((ntest,))
i=1
for i in np.arange(NFOLDS):    
    
    print(str(i+1)+"CatB.cbm")
    s=str(i+1)+"CatB.cbm"
    from_file = CatBoostRegressor()
    bst = from_file.load_model(s,format='cbm')
   
    predTr=bst.predict(test_set)
    predTest=predTest+predTr*wr
    i+=1

1CatB.cbm
2CatB.cbm
3CatB.cbm
4CatB.cbm
5CatB.cbm


In [80]:
preds = np.expm1(predTest)
preds

array([22.77577008, 13.17648618,  7.3964309 , ...,  6.10371447,
       36.35363723, 48.41114183])

In [81]:
preds = pd.DataFrame({"id": idx, "cost": preds})
preds.to_csv('cat.csv', index=False)